[Pagina del codigo](https://warwick.ac.uk/fac/sci/moac/people/students/peter_cock/python/genbank2fasta)

# GenBank to FASTA Nucleotides (*.gbk to *.fna)

In [ ]:
from Bio import SeqIO
import os


in_folder="in/"
out_folder="out/"

for f in os.listdir(in_folder):
    
    if not f.endswith(".gb") and not f.endswith(".gbk"):
        continue
        
    input_handle  = open(in_folder + f, "r")
    output_handle = open(out_folder+f.split('/')[-1:][0].split(".")[0]+".fasta", "w")

    for seq_record in SeqIO.parse(input_handle, "genbank") :
        output_handle.write(">%s %s\n%s\n" % (
               seq_record.id,
               seq_record.description,
               seq_record.seq))
                
    input_handle.close()
    output_handle.close()

print('Hecho')

# Gene by Gene : GenBank to FASTA Amino Acids (*.gbk to *.faa)

In [ ]:
from Bio import SeqIO
import os


in_folder="in/"
out_folder="out/"

for f in os.listdir(in_folder):
    
    if not f.endswith(".gb") and not f.endswith(".gbk"):
        continue
        
    input_handle  = open(in_folder + f, "r")
    output_handle = open(out_folder+f.split('/')[-1:][0].split(".")[0]+".fasta", "w")

    for seq_record in SeqIO.parse(input_handle, "genbank") :
        for seq_feature in seq_record.features :
            if seq_feature.type=="CDS" :
                assert len(seq_feature.qualifiers['translation'])==1
                output_handle.write(">%s from %s\n%s\n" % (
                       seq_record.id,
                       #seq_feature.qualifiers['locus_tag'][0],
                       seq_record.name,
                       seq_feature.qualifiers['translation'][0]))
                
    input_handle.close()
    output_handle.close()
    
print('Hecho')

# Gene by Gene : GenBank to FASTA Amino Acids (*.gbk to *.faa) V2

In [ ]:
from Bio import SeqIO
import os

in_folder="in/"
out_folder="out/"

for f in os.listdir(in_folder):
    
    if not f.endswith(".gb") and not f.endswith(".gbk"):
        continue
        
    input_handle  = open(in_folder + f, "r")
    
    for strand in [('ff'), ('rw')]:
            for frame in range(3):
                if os.path.exists(out_folder+f.split('/')[-1:][0].split(".")[0]+"_"+strand+"-"+str(frame)+".fasta"):
                    os.remove(out_folder+f.split('/')[-1:][0].split(".")[0]+"_"+strand+"-"+str(frame)+".fasta")

    for seq_record in SeqIO.parse(input_handle, "genbank") :
        seq = seq_record.seq
        table = 1
        min_pro_len = 400
        for strand, nuc in [('ff', seq), ('rw', seq.reverse_complement())]:
            for frame in range(3):
                pro_ok = False
                for pro in nuc[frame:].translate(table).split("*"):
                    if len(pro) >= min_pro_len:
                        pro_ok = True
                        print("%s => %s...%s - length %i, strand %s, frame %i => %s" % (seq_record.name, pro[:30], pro[-3:], len(pro), strand, frame,f.split('/')[-1:][0].split(".")[0]+"_"+strand+"-"+str(frame)+".fasta"))
                if pro_ok == True:
                    output_handle = open(out_folder+f.split('/')[-1:][0].split(".")[0]+"_"+strand+"-"+str(frame)+".fasta", "a")
                    output_handle.write(">%s from %s\n%s\n\n" % (
                           seq_record.id,
                           seq_record.name,
                           nuc[frame:].translate(table)))
                    output_handle.close()
                    
    input_handle.close()
    
    
print('Hecho')